# Tutorial 0 - Implementation of input information

## Configuration file
All the input information must be put in the correspondant configuration file,
organized in sections.

## Circuit scheme
This library aims to generate or analyze impedance vs frequency data, given
certain information about the system. The conceptual core structure is an
equivalent circuit scheme. It symbolizes the type and disposition of the
passive elements that represent the physical processes which rule the
impedance. Three (passive) elements types are considered: resistors,
capacitors and constant phase elements.

Any circuit can be schematized by a `diagram`, containing all its
elements. And upon this aspect lies the convention used in this code to
represent an electronic circuit. A passive element is symbolized by two
characters: a letter for its type followed by a single-digit number
to identify it. The convention adopted is letters `R`, `C` and `Q` for
resisitors, capacitors and constant phase elements, respectively, while the
numbers represent the order of apparition of the element in the diagram. A
circuit diagram must also contain the information about the specific
disposition of each element, where multiple elements can be in
parallel or in series. In the diagram, all the elements inside a pair of round
brackets `( )` are in series, while if they are inside a pair of square
brackets `[ ]` they are in parallel. Obviously, the order of elements inside a
pair of brackets is irrelevant, and usually I will write the resistors first,
just by convention. That being said, if we take a single resistor `R1` we can
add in serirs a capacitor `C2` just by writing `(R1C2)`.\
Speaking in terms of impedance, this will be equivalent to a circuit like

<p align="center">
    <img src="Images_tutorial/RC_series.png" alt="image" width="500"/>
</p>

In similar fashion, if we want the RC parallel circuit, we
may simply write `[R1C2]`. To go even further, a resistor that is in sereis
with an RC parallel circuit can be written as `(R1[R2Q3])`, and so on and so
forth. These two examples will represent these circuits (respectively)

<p align="center">
    <img src="Images_tutorial/RC_parallel.png" alt="image" width="460"/>
    &nbsp
    <img src="Images_tutorial/(R1[R2Q3]).png" alt="image" width="385"/>
</p>

Note that no whitespace is allowed in the diagram, and the letters are case
sensitive (i.e. they must be capital letters). Furthermore, the code accepts
only strings that begins with a bracket, and that of course are logically
consistent.\
The diagram must be put in the `[Circuit]` section of the configuration file.

## Parameters
Each element has its parameter(s) that describe it: resistors have the
resistance $R$, the capacitance $C$ and constant phase elements (CPE) have two
parameter: $Q$ and the ideality factor $n$. The first three must of course
be positive, while the last one spans between 0 and 1. The interpretation of
the constant phase element is not straightforward, but we can say that if $n$
is close to 1 the CPE behaves like a capacitor, whether if $n$ is close to 0
it is like a resisitor.

The contribution to the impedance for each elements depends on the value of
the parameter, since each element type is associated to an impedance formula:
- resistor $Z = R$
- capacitor $Z = \frac{1}{j \omega C}$
- constant phase elements $Z = \frac{1}{Q\,(2\pi \omega)^n\,e^{\frac{\pi}{2}nj}}$

where $Z$ is the impedance and $j$ is the imaginary unit.

The value of the set parameters must be specified inside `[Parameters]`
section of the configuration file, with the name of each element's name
followed by its value, in base units. This means that if we take the RC series
circuit `(R1C2)`, where the resistor has a resistance $R = 1\,k\Omega$ and the
capacitor a capacitance $C = 1\,\mu F$, the parameters will be

[Parameters]\
R1 = 1e3\
C2 = 1e-6

Instead, if we take the circuit `(R1[R2Q3])` with $R1 = 110\,\Omega$,
$R2 = 2000\,\Omega$ and for the CPE $Q = 10\,\mu F$ and $n = 0.5$, the
configuration settings are

[Parameters]\
R1 = 110\
C2 = 2200\
Q3 = 10e-6 0.5

Thus, the accepted parameters are a single number for `Rs` or `Cs` or 2
numbers separated by a whitespace for `Qs`. In order to have a working code,
there must be a good corrspondance betweem each parameter and its element in
the diagram.


## Constant elements conditions
Just for the analysis module, it is possible to set certain parameters as
constant (i.e. will not be considered in the minimization during the fit).\
This because, as said before, in certain ranges of frequency, the contribution
of some elements may be almost irrelevant, and thus it is better to just work
with the dominant elements if we have data in a limited range. To do so,
again, the constant condition is specified in the
`[Constant_parameter_conditions]` through the element's name and either `0`s
and `1`s, used here in a `True` or `False` fashion.

If we take the usual example `(R1C2)` and the constant conditions in the
configuration file are

[Constant_parameter_conditions]\
R1 = 0\
C2 = 0

it means that only the parameter of the resistor will be taken into
consideration during the miniminization. Note that the parameter of the
capacitor will still contribute to the overall impedance function, but for
the fit will be - obviously - constant.


## Frequency and data file settings
In the generation file the frequency points must be specified, to state how
many and in which frequency range the impedance data will be. In order to do
this, the maximum and minimum must be specified, as well as the number of
points desired. In particular must be stated the order of magnitude of the 
minimum tenth power and the maximum tenth power. For example, if we take
as minimum order of magnitude `1` and `5` as maximum, the smallest frequency
is $10\,Hz$ and the biggest is $100\,kHz$\
The data file name must be stated as well, putting it in the `[File]` section.
The only format considered is `.txt`. In the generation module it is also
possible to specify the format of the data that will be saved, in the same
section of the file's name. The only two formats accepted are either frequency
vs complex impedance (format = 2) or frequency vs modulus and impedance
(format = 3)


## Noise seed setting
In the generation module, a random error component is added to the signal to
better simulated experimental impedance data. For the sake of reproducibility
the random seed is set in the configuration file of the generation module
